In [166]:
import requests
import json

# api_server_url = "http://localhost:21122"
api_server_url = "https://allsecure.co.kr/llm"
# api_server_url = "http://localhost:41001"

# 1. Controller API

### GET /list_models 

In [169]:
ret = requests.get(api_server_url + "/list_models")

output_json = ret.json()
print(output_json)

{'models': [{'name': 'OLLM-Large', 'model_name': 'OLLM-Large_2023.11.20', 'description': '선택하신 모델은 OLLM-Large에요. OLLM-Large는 보다 질높은 답변을 하며, 다른 기술과 연동하여 긴 맥락으로부터 답변하는 상황에 유리해요'}]}


### GET /refresh_all_models

In [168]:
ret = requests.get(api_server_url + "/refresh_all_models")

### GET /get_random_uuid

In [132]:
ret = requests.get(api_server_url + "/get_random_uuid")

output_json = ret.json()
print(output_json)

{'uuid': '48a5a28aaf554537bcf9eaa8ebc64739'}


### (deprecated) {controller_address}/get_worker_address

model_name = output_json['models'][1]
print("model_name:", model_name)

input_json = {
    "model": model_name
}

ret = requests.post(
    controller_url + "/get_worker_address", 
    json=input_json
)

output_json = ret.json()
print(output_json)

# 2. Worker API

### POST /model_details 

In [133]:
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
}

ret = requests.post(
    api_server_url + "/model_details",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'context_length': 4096}


### POST /worker_get_conv_template  

In [124]:
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
}

ret = requests.post(
    api_server_url + "/worker_get_conv_template",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'conv': {'name': 'chat-orca', 'system_template': '{system_message}', 'system_message': '### System:\nYou are an AI assistant, please behave and help the user.', 'roles': ['### User', '### Assistant'], 'messages': [], 'offset': 0, 'sep_style': 2, 'sep': '\n\n', 'sep2': '</s>', 'stop_str': ['</s>'], 'stop_token_ids': None, 'tasks': {'retrieval': '### System:\nYou are an AI assistant, please behave and help the user.\nDate:{date}\nYour reply should be based on the context below.\n\nContext:{instruction}', 'instruct': '### System:\nYou are an AI assistant, please behave and help the user. Your reply should be based on the context below.\n\nContext:{instruction}', 'system_instruct': '### System:\n{system}', 'correction': '', 'summarization': '### System:\nYou are an AI assistant, Summarize below sentences.', 'enkotranslation': '### System:\nYou are an AI translator, who knows every language and how to translate one language to another. convert english sentences to korean sentences. do not 

### POST /count_token 

In [145]:
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
    # "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: NFC는 무엇의 약자입니까?\n\n### Assistant:"
    "prompt": """### System:
You are an AI assistant, please behave and help the user. Your name is OLLM(오름) by Ados(주식회사아도스), OLLM stands for On-premise LLM.
### User: I want you to act as a Table of Contents extractor. I will speak to you questions. I want you to reply with Table of Contents extracted from the questions. You must keep original text. do not write explanations. My first command is ```
〈목 차〉
제1장 서 론 ·································1
1.1 연구배경 ..............1
1.2 연구목적 ................2
제2장 연구의 범위 ··························3
2.1 VOC의 정의 및 중요성 .............3
2.2 관련 연구 및 차별성 .................3
제3장 자본이동론성 ·····························4
3.1 과거 자본론의 수집과 분류 ................5
3.2 자본이동론의 주제어 선정 .....................8
3.3 가중치의 설정 ........................... 13
3.4 온라인 자본 탐지 기법의 알고리즘 ........ 16
제4장 LO CB코인의 사회경제적 지위 ························18
4.1 실험에 사용된 데이터 ................ 21
4.2 실험 대상 및 결과 ...................... 22
제5장 새로운 방식의 보증개척 시스템-LO CB코인 23
제6장 맺음말 ·························24
참고문헌 ··············

차트 목차
<차트 1> 디지털 데이터 생성량 예측 추이표 ..............2
<차트 2> 전처리 파라미터 ............... 17
<차트 3> CB코인 사회지위 변화도 ........... 18
```

### Assistant: 〈목 차〉
제1장 서 론
1.1 연구배경
1.2 연구목적
제2장 연구의 범위
2.1 VOC의 정의 및 중요성
2.2 관련 연구 및 차별성
제3장 자본이동론성
3.1 과거 자본론의 수집과 분류
3.2 자본이동론의 주제어 선정
3.3 가중치의 설정
3.4 온라인 자본 탐지 기법의 알고리즘
제4장 LO CB코인의 사회경제적 지위
4.1 실험에 사용된 데이터
4.2 실험 대상 및 결과
제5장 새로운 방식의 보증개척 시스템-LO CB코인
제6장 맺음말
참고문헌</s>""",
}

ret = requests.post(api_server_url + "/count_token", json=input_json)

output_json = ret.json()
print(output_json)

{'count': 966, 'error_code': 0}


### POST /worker_generate

In [142]:
dummy = """Q: What command do I use to find the size of all the files (recursively) in a Linux or Mac OS X directory?\nA: The BSD version of ```du``` used in OS X reports size with 512-byte blocks -- the sizes are essentially rounded up to the next 512-byte value. This tells you the space on disk, which is larger than the amount of data. If you have a lot of small files, the difference can be large.\nHere's an example.\nThis is the value with regular ```du```. It's in 512-byte blocks:\n```$ du -s\n248   .\n```\nThe ```-h``` flag results in a more readable number, in kilobytes. As expected, it's half the number of 512-byte blocks:\n```$ du -hs\n124K  .\n```\nFinally, you can use ```find``` and ```awk``` to give you the sum of actual bytes in the files. This is kind of slow, but it works:\n```$ find . -type f -exec ls -l {} \\; | awk '{sum += $5} END {print sum}'\n60527\n```\nThis value matches exactly the number reported by Finder's Get Info window. (There are no weird forks or xattrs in this set of files.) It's significantly smaller than the value reported by ```du```.\nHere's how it works: it gets a list of all the files, and passes them to ```ls -l```; then ```awk``` is used to count up the bytes. The ```-type f``` flag is there so that only files (and not directories) get sent to ```ls```. Without that flag, it'll also send directory names to ```ls```, and each file will be listed twice : once as an individual file, and once as an item in the directory.\nThe GNU version of ```du``` can give values in actual bytes instead of blocks. It's unfortunate that the BSD version of ```du``` is not as flexible."""
prompt = f"### 영어:\n{dummy}\n### 한국어:\n"
input_json = {
    "model_name": "Gugugo-koen-7B-V1.1",
    "prompt": prompt,
    "temperature": 0.7,
    "top_p": 0.8,
    # "session_id": 3,
    "max_new_tokens": 4096,
    "stop": ["</끝>", "###"],
}

ret = requests.post(
    api_server_url + "/worker_generate",
    json=input_json,
)

output_json = ret.json()
# print(output_json)
output_json['text'][len(prompt):].rstrip('\n')

{'text': "### 영어:\nQ: What command do I use to find the size of all the files (recursively) in a Linux or Mac OS X directory?\nA: The BSD version of ```du``` used in OS X reports size with 512-byte blocks -- the sizes are essentially rounded up to the next 512-byte value. This tells you the space on disk, which is larger than the amount of data. If you have a lot of small files, the difference can be large.\nHere's an example.\nThis is the value with regular ```du```. It's in 512-byte blocks:\n```$ du -s\n248   .\n```\nThe ```-h``` flag results in a more readable number, in kilobytes. As expected, it's half the number of 512-byte blocks:\n```$ du -hs\n124K  .\n```\nFinally, you can use ```find``` and ```awk``` to give you the sum of actual bytes in the files. This is kind of slow, but it works:\n```$ find . -type f -exec ls -l {} \\; | awk '{sum += $5} END {print sum}'\n60527\n```\nThis value matches exactly the number reported by Finder's Get Info window. (There are no weird forks or 

### POST /worker_generate_stream

In [263]:
# input_json = {
#     "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
#     "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: react hello world\n\n### Assistant: 반응 프로젝트에서 \"Hello World\"를 표시하려면 다음 단계를 따르십시오.\n1. 새로운 React 응용 프로그램 생성 :\n먼저 새로운 React 응용 프로그램을 만들어야합니다.\n 다음 명령을 실행하여 새로운 React 앱을 만듭니다.\n```\nNPX Create-React-App My-App\n```\n이 명령은 새로운 폴더`My-App '에서 새로운 React 프로젝트를 만듭니다.\n2. 구성 요소 만들기 :\n이제 앱의 루트 구성 요소를 만들어야합니다.\n 터미널에서 다음 명령을 실행하여 새 구성 요소를 만듭니다.\n```\nCD My-App\nNPX React-S 생성 구성 요소 환경 적 응용 프로그램\n```\n이 명령은`src` 디렉토리에`myapp.js`라는 새 파���일을 생성합니다.\n3. 구성 요소 업데이트 :\n`myapp.js` 파일을 열고 다음 코드를 추가하십시오.\n```JS\n'React'에서 React React React;\n함수 myapp () {\n   반환 (\n     <h1> 안녕하세요, 세계! </h1>\n   );\n}\n수출 기본 MyApp;\n```\n이 코드는 \"Hello, World!\"텍스트를 표시하는 간단한 구성 요소를 만듭니다.\n4. 구성 요소 렌더링 :\n이제 앱의 기본 구성 요소를 렌더링해야합니다.\n `src/index.js` 파일을 열고 코드를 업데이트하십시오.\n```JS\n'React'에서 React React React;\n'./myapp'에서 myapp 가져 오기;\n기능 실행 () {\n   반환 (\n     <MyApp />\n   );\n}\n기본 기능 내보내기;\n```\n5. 앱 실행 :\n마지막으로 앱을 실행해보세요.\n```\nNPX 시작\n```\n이렇게하면 웹 브라우저에서 응용 프로그램을 실행하는 데 사용할 수있는 개발 서버가 시작됩니다.\n 응용 프로그램을 열고 브라우저에서 \"Hello, World!\"텍스트를 표시해야합니다.\n번역 번역\n반응 반응 프로젝트 프로젝트에서에서 \"hello world\"를 표시하려면하려면 다음",
#     "max_new_tokens": 1024,
#     "temperature": 0.7,
#     "top_p": 0.8,
#     "session_id": 3,
#     "repetition_penalty": 1.7,
# }
user_input = """
변화될 것이며, 데이터 량 기준은 산업분야에 따라 상대적이다.’ 라고 정
의 하였다[4].
이 데이터들의 활용은 개별적이고, 즉각적이며, 다면적인 검토를 거
친 부가가치를 제공해야 한다. 이 관점에서 빅 데이터는 피드백 대상과
실시간성 이라는 두 방향에서 바라볼 필요가 있다. 우선 피드백 대상은
데이터를 처리하고 분석하여 얻은 가치(Value)의 피드백 대상이 개별적
인 유저인지 아니면 서비스 이용자 전체인지 여부이다. 그 다음의 실시
간성은 과거에서 부터 누적되어 왔던 데이터를 분석하여 처리결과를 반
환해주는 과거 축적 형인지 아니면 실시간으로 생성되는 데이터들을 분
석하여 결과를 반환해주는 실시간 형인지의 여부이다[16]. 초기의 데이
터 분석은 과거 축적데이터를 분석하여 계열전체에 피드백을 하는 연구
가 대부분 이었지만, 실시간으로 쏟아져 나오는 다양한 출처의 데이터
증가와 함께 최근에는 실시간으로 데이터를 분석하여 개별적으로 피드백
을 제공하는 연구들이 이루어지고 있다.
과거 축적 형
(피드백 오래 걸림)
실시간 형
(즉시 피드백 가능)
계열전체에
피드백
소매업 진열대의
상품 배열 최적화
방범 카메라 영상의
실시간 자동 이상탐지
개별적으로
피드백
전자상거래 사이트의
개인별 상품추천
VISA 카드의 사용자
부정패턴 실시간 탐지
[표 2] 피드백 대상과 실시간성
- 5 -
최근 온라인 이용자의 급격한 증가와 함께 늘어나는 데이터들을 실
시간으로 감지(Real Time Sensing)하는 방법들이 다양한 산업 분야에서
활용되고 있다.
코카콜라(Coca-Cola)는 전 세계 여러 나라의 트위터 이용자들이
온라인으로 작성하는 글들을 실시간으로 분석하여, 코카콜라에 대한 부
정적인 단어(Keyword)가 증가하는 국가나 지역을 대상으로 집중적인
홍보를 실시한다. 코카콜라의 트위터 분석은 영어, 중국어, 일본어, 한국
어, 아랍어 등 세계 각 나라의 다양한 언어로 분석되고 있으며, 분석되
어진 정보는 각 나라의 코카콜라 자회사로 제공된다.
메디시스(MedISys)는 전 세계의 인터넷 문서들을 자동으로 검색하
고, 문서 내부의 단어(Keyword)들을 분석하여 공중보건과 관련된 질병
이 발생할 것으로 예측되는 지역에 대해 알림을 준다. 메디시스는 43개
국의 언어를 분석하며, 1400여개의 뉴스 포털에서 하루 5만개의 정도의
기사들을 분석하고 있다.
구글(Google)은 독감 증상이 있는 사람들이 늘어나면 독감과 관련
된 단어(Keyword)의 검색들이 늘어나는 통계를 기초로 하여, 시간 및
지역별 독감 유행 정보를 제공하고 있다. 미국 보건당국은 예보는 일주
일 단위로 정보를 알려주지만, 구글은 매일 정보를 알려주기 때문에 보
건당국보다 더 빠르게 독감 유행의 징후를 감지하고, 대처할 수 있도록
지원한다[12]. 구글은 2008년 2월 미국질병통제센터(CDC)가 Atlanta
- 6 -
주 정부의 독감 발생을 예보한 것보다, 2주 더 빨리 독감 발생을 예보한
바가 있다[13].
국내에서는 스마트 폰 출시 전에 소비자들이 해당 스마트폰에 대해
어떤 의견들을 가지고 있는지를 댓글 분석을 통해 분석하고, 듀얼 코어
로 인해 배터리 시간이 짧을 것이라는 부정적 정보를 미리 파악하여 마
케팅 커뮤니케이션에 활용하기도 하였다[18].
과거 20년에서 25년 동안 기업들은 활용이 가능한 정보의 약 5%만
을 의사결정에 활용해왔다[17]. 빅 데이터의 분석은 많다(Big), 적다
(Small) 라는 데이터 규모의 이야기에서 끝나는 것이 아니라, 다양한 정
형, 비정형, 반정형 데이터들 속에서 데이터의 패턴을 찾아내고, 그 패턴
의 규칙을 어떻게 처리해서 가치(Value)있는 통찰력(Insight)를 얻어낼
수 있는가 하는 것을 포함하고 있어야 한다. 즉, 빅 데이터란 큰 규모
(Big Volume)만을 의미하기 보다는 ‘큰 가치(Big Value)를 얻을 수 있
는 규모’ 라는 상대적인 개념으로 받아들여야 하며, 빅 데이터의 가치
(Value)는 패턴을 찾아내고 분석하는 능력과 그 결과로 얻어진 통찰력
(Insight)으로 데이터를 처리하는 능력에 달려 있다고 할 수 있다.
1.2 연구목적
VOC(Voice of Customer)의 활용은 기업의 고객중심 경영에 매우
- 7 -
중요한 역할을 하며, 그 중 긴급한 VOC 게시물을 신속하게 확인하고 처
리하는 것은 기업의 수익과 직접적이고도 밀접한 관련이 있다. 하지만
온라인 상담 게시판 및 상품평 게시판을 통해 파악되는 게시물은 운영자
가 해당 게시물을 읽고 상황을 파악하기까지 많은 시간을 필요로 하게
된다. 보통 50여명 규모의 기업의 운영하는 온라인 컨텐츠 제공업체의
경우 고객 서비스 팀에 접수된 게시물은 고객이 온라인에 글을 작성 한
후 평균 1시간 이후에 고객이 작성한 게시물을 읽어보게 되며, 고객이
많이 몰리는 시간대에는 2-3시간 이후에 고객이 올린 상담 게시물을 읽
고 상황을 파악하게 된다. 또한 상품평란에 작성된 고객의 코멘트는 1일
이 훨씬 지난 이후에 파악하게 되는 경우도 있다.
본 논문에서는 기업의 온라인 웹페이지에서 작성되는 게시물의 긴급
성 여부를 실시간으로 파악해 긴급 게시물일 경우 운영자에게 바로 알림
을 주어 빠르게 대처할 수 있도록 하여, 시스템 운용 프로세스를 향상시
킬 수 있도록 하는 텍스트 마이닝을 이용한 온라인 긴급 게시물 실시간
탐지 기법을 제안한다.
이를 위해 먼저 과거 축적된 VOC 게시물 데이터 중 긴급 게시물과
비긴급 게시물을 분류 한 후, 긴급 게시물 데이터들에서 긴급 게시물의
주제어를 찾아낸다. 그리고 긴급 게시물 데이터와 비긴급 게시물 데이터
에서 각각 긴급 게시물 주제어를 서술하는 서술어를 모두 추출한다. 그
리고 동일한 서술어로 서술되어지는 긴급 게시물의 주제어를 그룹화 한
"""

instruct = (
    "I want you to act as a Table of Contents extractor. "
    "I will speak to you questions. "
    "I want you to reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text."
    "And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. "
    "do not write explanations. My first command is "
)
prompt = f"""### System:
You are an AI assistant, please behave and help the user.
### User: {instruct}```
{user_input}
...
```

### Assistant:"""



input_json = {
    "model_name": "OLLM-Small_2024.01.16", #"OLLM-Large_2023.11.20", # , #"DIE_10.7b_v0.1_ep2","DIE-MoE-10.7Bx4_v8_gpt_ep3"
    "prompt": prompt,
    "temperature": 0.7,
    "top_p": 0.8,
    # "session_id": 3,
    "max_new_tokens": 4096,
    # "stop": ["</끝>", "###"],
}

ret = requests.post(
    api_server_url + "/worker_generate_stream",
    json=input_json,
    stream=True,
    # timeout=200,
)

past_len = len(input_json["prompt"])
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

# for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
#     if chunk:
#         data = json.loads(chunk.decode())
#         # print(data["text"][past_len:], sep="", end="")
#         # past_len = len(data["text"])

# result = json.loads(chunk.decode())

 [목차(Table of Contents)]

1. 서론
2. 연구 배경
2.1 빅 데이터(Big Data)
2.2 VOC(Voice of Customer)
2.3 온라인 긴급 게시물 실시간 탐지
3. 연구 목적
4. 연구 방법
4.1 데이터 수집
4.2 데이터 전처리
4.3 텍스트 마이닝
4.4 기계 학습
4.5 실시간 탐지 시스템
5. 결과
6. 토론
7. 결론
8. 참고 문헌

[dotted line, page number, 제목(title), content, explanation, summary, predicted] 없음.

In [157]:
data['text']

"### 영어:\nQ: What command do I use to find the size of all the files (recursively) in a Linux or Mac OS X directory?\nA: The BSD version of ```du``` used in OS X reports size with 512-byte blocks -- the sizes are essentially rounded up to the next 512-byte value. This tells you the space on disk, which is larger than the amount of data. If you have a lot of small files, the difference can be large.\nHere's an example.\nThis is the value with regular ```du```. It's in 512-byte blocks:\n```$ du -s\n248   .\n```\nThe ```-h``` flag results in a more readable number, in kilobytes. As expected, it's half the number of 512-byte blocks:\n```$ du -hs\n124K  .\n```\nFinally, you can use ```find``` and ```awk``` to give you the sum of actual bytes in the files. This is kind of slow, but it works:\n```$ find . -type f -exec ls -l {} \\; | awk '{sum += $5} END {print sum}'\n60527\n```\nThis value matches exactly the number reported by Finder's Get Info window. (There are no weird forks or xattrs in

In [14]:
print("""Node.js 애플리케이션에서 파일을 수신하려면 Express 프레임워크를 사용하여 간단한 웹 서버를 만들 수 있습니다. 다음 예제에서는 `express` 및 `multer` 라이브러리를 사용하여 파일 업로드를 수신하는 방법을 보여줍니다.\n\n1. 먼저 npm을 사용하여 필요한 종속성을 설치합니다:```bash\nnpm init -y\nnpm install express multer\n```\n\n2. 새 JavaScript 파일을 생성하고 다음 코드를 붙여넣습니다(예: `app.js`).```javascript\nconst express = require(\'express\');\nconst multer = require(\'multer\');\n\nconst app = express();\nconst port = 3000;\n\n// Configure multer storage settings\nconst storage = multer.diskStorage({\n  destination: (req, file, cb) => {\n    cb(null, \'uploads/\');\n  },\n  filename: (req, file, cb) => {\n    cb(null, `${file.fieldname}-${Date.now()}${getExtension(file.mimetype)}`);\n  },\n});\n\nconst upload = multer({ storage });\n\nfunction getExtension(mimetype) {\n  const parts = mimetype.split(\'/\');\n  return parts[parts.length - 1];\n}\n\n// ROUTES\napp.post(\'/upload\', upload.single(\'file\'), (req, res) => {\n  res.send(`File received: ${req.file.filename}`);\n});\n\n// Start the server\napp.listen(port, () => {\n  console.log(`Server is running at http://localhost:${port}`);\n});\n```\n\n\n이 코드는 포트 3000에서 수신 대기하고 파일 업로드를 위한 단일 경로(`/upload`)를 가진 간단한 Express 서버를 설정합니다. 파일은 \'uploads/\' 폴더에 저장됩니다. app.js` 파일과 같은 디렉터리에 \'uploads/\' 폴더를 생성했는지 확인하세요.\n\n3. 서버를 실행합니다:```bash\nnode app.js\n```\n\n4. Postman 또는 CURL과 같은 도구를 사용하여 파일 업로드를 테스트합니다:```bash\ncurl -X POST -H "Content-Type: multipart/form-data" -F "file=@/path/to/yourfile.txt" "http://localhost:3000/upload"\n```\n\n\n"경로/대상/파일.txt"를 업로드하려는 파일의 경로로 바꿉니다.\n\n이제 서버는 파일을 수신하여 \'업로드/\' 폴더에 저장하고 업로드된 파일의 이름으로 메시지를 반환해야 합니다.""")

Node.js 애플리케이션에서 파일을 수신하려면 Express 프레임워크를 사용하여 간단한 웹 서버를 만들 수 있습니다. 다음 예제에서는 `express` 및 `multer` 라이브러리를 사용하여 파일 업로드를 수신하는 방법을 보여줍니다.

1. 먼저 npm을 사용하여 필요한 종속성을 설치합니다:```bash
npm init -y
npm install express multer
```

2. 새 JavaScript 파일을 생성하고 다음 코드를 붙여넣습니다(예: `app.js`).```javascript
const express = require('express');
const multer = require('multer');

const app = express();
const port = 3000;

// Configure multer storage settings
const storage = multer.diskStorage({
  destination: (req, file, cb) => {
    cb(null, 'uploads/');
  },
  filename: (req, file, cb) => {
    cb(null, `${file.fieldname}-${Date.now()}${getExtension(file.mimetype)}`);
  },
});

const upload = multer({ storage });

function getExtension(mimetype) {
  const parts = mimetype.split('/');
  return parts[parts.length - 1];
}

// ROUTES
app.post('/upload', upload.single('file'), (req, res) => {
  res.send(`File received: ${req.file.filename}`);
});

// Start the server
app.listen(port, () => {
  console.log(`Server is

### POST /worker_generate_auto

In [147]:
input_json = {
    "model_name": "Gugugo-koen-7B-V1.1",
    "user_input": """Q: What command do I use to find the size of all the files (recursively) in a Linux or Mac OS X directory?\nA: The BSD version of ```du``` used in OS X reports size with 512-byte blocks -- the sizes are essentially rounded up to the next 512-byte value. This tells you the space on disk, which is larger than the amount of data. If you have a lot of small files, the difference can be large.\nHere's an example.\nThis is the value with regular ```du```. It's in 512-byte blocks:\n```$ du -s\n248   .\n```\nThe ```-h``` flag results in a more readable number, in kilobytes. As expected, it's half the number of 512-byte blocks:\n```$ du -hs\n124K  .\n```\nFinally, you can use ```find``` and ```awk``` to give you the sum of actual bytes in the files. This is kind of slow, but it works:\n```$ find . -type f -exec ls -l {} \\; | awk '{sum += $5} END {print sum}'\n60527\n```\nThis value matches exactly the number reported by Finder's Get Info window. (There are no weird forks or xattrs in this set of files.) It's significantly smaller than the value reported by ```du```.\nHere's how it works: it gets a list of all the files, and passes them to ```ls -l```; then ```awk``` is used to count up the bytes. The ```-type f``` flag is there so that only files (and not directories) get sent to ```ls```. Without that flag, it'll also send directory names to ```ls```, and each file will be listed twice : once as an individual file, and once as an item in the directory.\nThe GNU version of ```du``` can give values in actual bytes instead of blocks. It's unfortunate that the BSD version of ```du``` is not as flexible.""",
    "session_id": 20,
    "temperature": 0.7,
    "top_p": 0.8,
    "max_new_tokens": 4096,
    "stop": ["</끝>", "###"],
}

ret = requests.post(
    api_server_url + "/worker_generate_auto",
    json=input_json,
)

output_json = ret.json()
output_json['text']

In [139]:
print(output_json['text'])

Q: Linux 또는 Mac OS X 디렉터리에 있는 모든 파일의 크기를 재귀적으로 찾으려면 어떤 명령을 사용해야 하나요?
A: OS X에서 사용되는 BSD 버전의 ```du```는 512바이트 블록 단위로 크기를 보고하며, 크기는 기본적으로 다음 512바이트 값으로 반올림됩니다. 따라서 디스크 공간은 데이터의 양보다 더 큰 양입니다. 작은 파일이 많은 경우 그 차이가 클 수 있습니다.
다음은 예시입니다.
이것은 일반적으로 ```du```를 사용하는 값입니다. 512바이트 블록 단위로 표시됩니다:
```$ du -s
248 .
```
du -h``` 플래그를 사용하면 킬로바이트 단위로 더 읽기 쉬운 숫자가 표시됩니다. 예상대로 512바이트 블록의 절반 정도입니다:
```$ du -hs
124K .
```
마지막으로, ```find``` 및 ```awk```를 사용하여 파일에 있는 실제 바이트의 합계를 구할 수 있습니다. 이 방법은 속도가 느리지만 작동합니다:
```$ find . -type f -exec ls -l {} \; | awk '{sum += $5} END {print sum}'
60527
```
이 값은 Finder의 정보 확인 창에 표시되는 수치와 정확히 일치합니다. (이 파일 세트에는 이상한 포크나 xattr이 없습니다.) 이 값은 ```du```가 보고하는 값보다 훨씬 작습니다.
이 방법은 모든 파일 목록을 가져와서 ```ls -l```로 전달한 다음 ```awk```를 사용하여 바이트 수를 계산하는 것입니다. type f``` 플래그는 파일(디렉터리 제외)만 ```ls```로 전송되도록 하기 위해 사용됩니다. 이 플래그가 없으면 디렉터리 이름도 ```ls```로 전송되며, 각 파일은 개별 파일로 한 번, 디렉터리의 항목으로 한 번 등 두 번 나열됩니다.
du```의 GNU 버전은 블록 대신 실제 바이트 단위로 값을 제공할 수 있습니다. 안타깝게도 BSD 버전의 ```du```는 유연성이 떨어집니다.



### POST /worker_generate_stream_auto

In [146]:
input_json = {
    "model_name": "Gugugo-koen-7B-V1.1",
    "user_input": """Q: What command do I use to find the size of all the files (recursively) in a Linux or Mac OS X directory?\nA: The BSD version of ```du``` used in OS X reports size with 512-byte blocks -- the sizes are essentially rounded up to the next 512-byte value. This tells you the space on disk, which is larger than the amount of data. If you have a lot of small files, the difference can be large.\nHere's an example.\nThis is the value with regular ```du```. It's in 512-byte blocks:\n```$ du -s\n248   .\n```\nThe ```-h``` flag results in a more readable number, in kilobytes. As expected, it's half the number of 512-byte blocks:\n```$ du -hs\n124K  .\n```\nFinally, you can use ```find``` and ```awk``` to give you the sum of actual bytes in the files. This is kind of slow, but it works:\n```$ find . -type f -exec ls -l {} \\; | awk '{sum += $5} END {print sum}'\n60527\n```\nThis value matches exactly the number reported by Finder's Get Info window. (There are no weird forks or xattrs in this set of files.) It's significantly smaller than the value reported by ```du```.\nHere's how it works: it gets a list of all the files, and passes them to ```ls -l```; then ```awk``` is used to count up the bytes. The ```-type f``` flag is there so that only files (and not directories) get sent to ```ls```. Without that flag, it'll also send directory names to ```ls```, and each file will be listed twice : once as an individual file, and once as an item in the directory.\nThe GNU version of ```du``` can give values in actual bytes instead of blocks. It's unfortunate that the BSD version of ```du``` is not as flexible.""",
    "session_id": 22,
    # "context": '\nDate:2023-11-13 15:21\nYour reply should be based on the context below.\n\nContext:\n(주)아도스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도스 기업소개 - 업력 : 10년차, 기업형태 : 중소기업, 업종 : 응용 소프트웨어 개발 및 공급업 | (주)아도스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 ... 재산, 가치를 구하는 것을 사명으로, 좋은 사람들이 모여 즐겁게 성장하고 있는 회사입니다.\n1. 현재는 IT보안이 주 사업 영역이며, AI와 가상화기술 기반의 정보보호 솔루션을 개발/운영하고 있습니다.\n2. 대기업 제조사의 네트워크 보안, 생산라인 보안, 정보 유출 방지 등 통합보안 서비스(모의 해킹, 진단, 관제, 이상징후 분석)를 담당하고 있습니다.\n"인류의 생명과 가치를 구한다"\n우리는 단단하게 성장하고 있어요\n-\n2021\n-\n09월\n가드랜드(정보보호솔루션) 출시\n-\n03월\n가두리(웹격리솔루션) 출시\n- 09월\n-\n2020\n\n-\n09월\n병역지정업체(산업체) 선정\n-\n05월\n벤처기업확인(기술보증기금,유효기간:2022.05.28)\n-\n04월\n수차에 걸쳐 자본금을 1200백만원으로 증자\n- 09월\n-\n2019\n-\n12월\n기업부설연구소 설립\n- 12월\n-\n2017\n-\n10월\n상호를 (주)아도스로 변경\n- 10월\n-\n2014\n-\n09월\n컴퓨터 프로그래밍 서비스업을 목적으로 대표이사 이승원에 의해 (주)피그플라이 설립\n- 09월\n우리는 직원의 복리후생도 중요해요\n-\n지원금/보험\n- 각종 경조사 지원\n-\n급여제도\n- 퇴직연금\n- 인센티브제\n- 스톡옵션\n- 퇴직금\n-\n업\n- 대표자명\n- 이승원\n- 사업내용\n-\n소프트웨어 개발,게임 개발\n- 주소\n-\n서울 송파구 송파대로 167, B동 1021호지도보기\n- SNS\n-\n- 보유기술\n-\n-\n특허권\n- 디스플레이 해킹 방지 시스템 및 그 방법(2020)\n-\n특허권\n- 웹 클라이언트를 이용한 망 분리 시스템 및 그 방법(2020)\n-\n특허권\n- 컴퓨터의 시스템자원 및 프로세스의 보호 및 격리장치와 그방법(2020)\n-\n특허권\n- PKI 기반의 사물인터넷 기기 인증방법 및 인증시스템(2020)\n- 특허권\n미래 혹은 가까에 있는 위협으로 부터, 우리의 생명,\n\n\n(주)아도니스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도니스 기업소개 - 업력 : 35년차, 기업형태 : -, 업종 : 골프장 운영업 | (주)아도니스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 궁금하시다면, ...- 사업내용\n-\n골프장,호텔 운영/무역,담배,골프용품 도소매,통신판매/한식,중식 음식점\n- 주소\n-\n경기 포천시 신북면 포천로 2499지도보기\n기업비전\n주요 사업내용 : 포천 아도니스CC 운영\n주요 사업내용 : 포천 아도니스CC 운영\n주요 취급품목 : 골프장운영, 호텔운영\n우리는 단단하게 성장하고 있어요\n-\n2005\n-\n01월\n호텔 준공 영업개시\n- 01월\n-\n1999\n-\n06월\n상호변경 : (주)대우레져 -> (주)아도니스\n-\n04월\n골프장 개장\n- 06월\n-\n1995\n-\n04월\n골프장 착공허가 (경기도 포천군)\n- 04월\n-\n1\n990\n-\n10월\n경기도로부터 골프장사업계획을 승인\n- 10월\n-\n1989\n-\n02월\n회사 설립\n- 02월\n인원 규모를 파악해 보세요!\n전체 직원 수 56명\n- 전체인원\n- 입사자\n- 퇴사자\n',
    "max_new_tokens": 4096,
    "temperature": 0.7,
    "top_p": 0.8,
}

ret = requests.post(
    api_server_url + "/worker_generate_stream_auto",
    json=input_json,
    stream=True,
    # timeout=2,
)

# past_len = len(input_json["prompt"])
# past_len = 0
# for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
#     if chunk:
#         data = json.loads(chunk.decode())
#         print(data["text"][past_len:], sep="", end="")
#         past_len = len(data["text"])

### POST /worker_get_last_prompt

In [11]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_get_last_prompt",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'prompt': '### System:\nThis is a system prompt, please behave and help the user.\nDate:2023-11-13 15:21\nYour reply should be based on the context below.\n\nContext:\n(주)아도스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도스 기업소개 - 업력 : 10년차, 기업형태 : 중소기업, 업종 : 응용 소프트웨어 개발 및 공급업 | (주)아도스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 ... 재산, 가치를 구하는 것을 사명으로, 좋은 사람들이 모여 즐겁게 성장하고 있는 회사입니다.\n1. 현재는 IT보안이 주 사업 영역이며, AI와 가상화기술 기반의 정보보호 솔루션을 개발/운영하고 있습니다.\n2. 대기업 제조사의 네트워크 보안, 생산라인 보안, 정보 유출 방지 등 통합보안 서비스(모의 해킹, 진단, 관제, 이상징후 분석)를 담당하고 있습니다.\n"인류의 생명과 가치를 구한다"\n우리는 단단하게 성장하고 있어요\n-\n2021\n-\n09월\n가드랜드(정보보호솔루션) 출시\n-\n03월\n가두리(웹격리솔루션) 출시\n- 09월\n-\n2020\n\n-\n09월\n병역지정업체(산업체) 선정\n-\n05월\n벤처기업확인(기술보증기금,유효기간:2022.05.28)\n-\n04월\n수차에 걸쳐 자본금을 1200백만원으로 증자\n- 09월\n-\n2019\n-\n12월\n기업부설연구소 설립\n- 12월\n-\n2017\n-\n10월\n상호를 (주)아도스로 변경\n- 10월\n-\n2014\n-\n09월\n컴퓨터 프로그래밍 서비스업을 목적으로 대표이사 이승원에 의해 (주)피그플라이 설립\n- 09월\n우리는 직원의 복리후생도 중요해요\n-\n지원금/보험\n- 각종 경조사 지원\n-\n급여제도\n- 퇴직연금\n- 인센티브제\n- 스톡옵션\n- 퇴직금\n-\n업\n- 대표자명\n- 이승원\n- 사업내용\n-\n소프

### POST /session_history_clear

In [12]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/session_history_clear",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'err_code': 0}


### POST /worker_regenerate_auto

In [13]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_regenerate_auto",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'err_code': 70003}


### POST /worker_regenerate_stream_auto

In [14]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_regenerate_stream_auto",
    json=input_json,
    stream=True,
)

# past_len = len(input_json["prompt"])
past_len = 0
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        if "text" not in data:
            continue
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

### POST /worker_stop_stream

In [15]:
import uuid
from multiprocessing import Process
import time


# background runner for worker_generate_stream
def background_run_stream(session_id):
    input_json = {
        "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
        "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: 유튜브에 대해서 설명해주세요.\n\n### Assistant:",
        "session_id": session_id,
        "temperature": 0.7,
        "top_p": 0.8,
    }

    ret = requests.post(
        api_server_url + "/worker_generate_stream",
        json=input_json,
        headers={"User-Agent": "FastChat Client"},
        stream=True,
        timeout=200,
    )

    past_len = len(input_json["prompt"])
    for idx, chunk in enumerate(ret.iter_lines(decode_unicode=False, delimiter=b"\0")):
        if chunk:
            data = json.loads(chunk.decode())
            print(data["text"][past_len:], end="")
            past_len = len(data["text"])


# generate random session_id
session_id = uuid.uuid4().hex
print("session_id:", session_id)

# run worker_generate_stream
p1 = Process(target=background_run_stream, args=[session_id])
p1.start()

# wait for 3 seconds
time.sleep(3)

# send worker_stop_stream
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": session_id,
}

ret = requests.post(
    api_server_url + "/worker_stop_stream",
    json=input_json,
)

print("\n\n Generation Stopped")

session_id: b5bdc37a9ed14ac1b3d05e7ff5566659
 유튜브는 세계에서 가장 큰 온라인 동영상 공유 및 소셜 미디어 플

 Generation Stopped
